# **01: Sequential Chains**


LangChain allows developers to build chains of prompts and models. A sequential chain connects multiple components where the output of one becomes the input of the next.

This is useful when you want to perform multi-step operations, such as planning a trip itinerary.



## AI-Powered Travel Recommendations


In [ ]:
# Import required components
from langchain.prompts import PromptTemplate                      # For formatting prompts with variables
from langchain_core.output_parsers import StrOutputParser         # Extracts plain text output from LLM responses for easier processing
from langchain_google_genai import ChatGoogleGenerativeAI         # Gemini model via LangChain
from langchain_core.runnables import RunnableMap                  # # Maps multiple inputs to corresponding outputs in a chain for parallel execution
from dotenv import load_dotenv                                    # To load API keys from .env

# Load environment variables (like Gemini API key)
load_dotenv()

# Prompt to get activity suggestions based on destination
destination_prompt = PromptTemplate(
    input_variables=["destination"],
    template="I am planning a trip to {destination}. Can you suggest some activities to do there?"
)

# Prompt to generate itinerary from suggested activities
activities_prompt = PromptTemplate(
    input_variables=["activities"],
    template="I only have one day, so can you create an itinerary from your top three activities: {activities}."
)

# Initialize Gemini model with desired settings
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",          # Specify the model to use
    # max_output_tokens=100,           # Optional: limit response length
    temperature=0.7                    # Controls creativity of output
)

# Parser to extract clean string output from LLM
parser = StrOutputParser()

# Step 1: Generate activity list from destination
step1 = destination_prompt | llm | parser

# Step 2: Feed activities into itinerary prompt, then get final response
seq_chain = RunnableMap({"activities": step1}) | activities_prompt | llm | parser

# Run the chain with an example input
output = seq_chain.invoke({"destination": "New Delhi"})
print(output)

Given you only have one day, here's a jam-packed itinerary focusing on three key experiences offering a blend of history, culture, and a taste of Delhi's energy:

**Morning (Historical Grandeur):**

* **8:00 AM - 11:00 AM: Red Fort (Lal Qila):** Start your day early at the magnificent Red Fort.  Allow at least 3 hours to explore this UNESCO World Heritage site thoroughly.  Consider hiring a guide for a richer experience (optional, but highly recommended).

**Afternoon (Cultural Immersion & Relaxation):**

* **11:30 AM - 1:30 PM:  Rickshaw Ride through Old Delhi & Jama Masjid:** Take a thrilling rickshaw ride through the bustling streets of Old Delhi to reach Jama Masjid.  Negotiate the rickshaw fare beforehand.  Spend some time admiring the architecture of Jama Masjid (remember respectful attire).

* **1:30 PM - 2:30 PM: Lunch:** Enjoy a delicious and authentic Delhi street food lunch near Jama Masjid.  There are plenty of options, but choose carefully considering hygiene.

* **2:30 PM

## Building Smart AI-Powered Learning Assistants (Personalized)

In [ ]:
# Import required components
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser 
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.runnables import RunnableMap
from dotenv import load_dotenv

# Load environment variables (like Gemini API key)
load_dotenv()

# Prompt to get learning resources based on topic
learning_prompt = PromptTemplate(
    input_variables=["topic"],
    template="I want to learn about {topic}. Can you suggest some resources like articles, videos, or courses?"
)

# Prompt to generate a personalized study plan using resources + user info
study_plan_prompt = PromptTemplate(
    input_variables=[
        "resources",
        "programming_experience",
        "math_background",
        "learning_style",
        "learning_goals",
        "time_commitment"
    ],
    template= """
    I have the following resources: {resources}
    
    Create a personalized study plan using these resources based on this user's information:
    
        1. Programming experience: {programming_experience}
        2. Math background: {math_background}
        3. Preferred learning style: {learning_style}
        4. Learning goals: {learning_goals}
        5. Time commitment per week: {time_commitment} hours

        Make the plan structured and practical. Include weeks/modules, brief tasks, and align them with the user's preferences and pace.
        """
        )

# Initialize Gemini model
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.7
)

# Output parser
parser = StrOutputParser()

# Step 1: Generate resource list from topic
step1 = learning_prompt | llm | parser

# Step 2: Personalized study plan using resources and user profile
seq_chain = RunnableMap({
    "resources": step1,
    "programming_experience": lambda x: x["programming_experience"],
    "math_background": lambda x: x["math_background"],
    "learning_style": lambda x: x["learning_style"],
    "learning_goals": lambda x: x["learning_goals"],
    "time_commitment": lambda x: x["time_commitment"],
}) | study_plan_prompt | llm | parser

# Example input from user
user_input = {
    "topic": "Machine Learning",
    "programming_experience": "Intermediate (Python, Pytorch, TensorFlow)",
    "math_background": "Intermediate (Calculus, Linear Algebra)",
    "learning_style": "Hands-on projects and video lectures",
    "learning_goals": "Build a portfolio to get a job in data science",
    "time_commitment": "10"
}

# Run the chain
output = seq_chain.invoke(user_input)
print(output)

## Personalized Machine Learning Study Plan (10 hours/week)

This plan prioritizes hands-on projects and video lectures, leveraging the user's intermediate Python, PyTorch/TensorFlow, and math background.  The goal is portfolio building for a data science job.

**Phase 1: Solidify Foundations & Practical Skills (Weeks 1-4)**

* **Module 1: Regression & Classification (Week 1-2):**
    * **Task 1 (2 hours):** Review linear algebra & calculus concepts relevant to ML (Khan Academy or 3Blue1Brown for focused review).
    * **Task 2 (4 hours):** Complete DataCamp's introductory machine learning course focusing on regression and classification.  Focus on practical implementation in Python (Scikit-learn).
    * **Task 3 (4 hours):**  Build a simple regression project (e.g., house price prediction using a publicly available dataset).  Use Scikit-learn, document your code thoroughly, and analyze results.

* **Module 2:  Deep Learning Introduction (Week 3-4):**
    * **Task 1 (3 hours):** Watch 

# 02: LangChain Agents and ReAct

* **Agents in LangChain**:

  * Use LLMs to make decisions and perform actions by calling external tools.
  * Enable dynamic workflows where the model can choose the next step based on the task.

* **ReAct Pattern** (Reasoning + Acting):

  * A framework where the LLM thinks through a problem step by step.
  * Chooses a relevant tool to act with.
  * Observes the tool's output.
  * Continues reasoning based on the observed result.

* **Flow of ReAct Agent**:

  1. **Thought**: Model decides what to do next.
  2. **Act**: Model selects and calls the appropriate tool with input.
  3. **Observe**: Model receives and processes the tool's output.
  4. **Repeat or respond**: Model may continue thinking and acting or provide the final answer.

* **Example**:

```
# Thought: I should call Weather() to find the weather in Kingston, Jamaica.
# Act: Weather("Kingston, Jamaica")
# Observe: The weather is mostly sunny with temperatures of 82°F.
```




# 03: LangGraph


* 🔄 **LangGraph** is a framework built on top of LangChain.
* 🧠 It helps you create **stateful AI agents** — ones that remember and adapt as they go.
* 🔧 Based on **graphs**, where each node is a tool or step in the agent's logic.
* 🔁 Supports **loops and branching**, so the agent can revisit steps or make decisions.
* 🧱 Makes building **complex workflows** with language models easier and more structured.
* 🤝 Works well with tools, memory, and custom logic for smarter agents.
* ⚡ Good for tasks like **chatbots, planning agents, and multi-step reasoning.**

## Building a Gemini-Based Agent for Math Calculations

In [21]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.agent_toolkits.load_tools import load_tools
from langchain.agents import initialize_agent
from langchain.agents.agent_types import AgentType

# Load environment and setup
from dotenv import load_dotenv
load_dotenv()

# Initialize Gemini LLM
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash",
                             temperature=0.7)

# Load math tool
tools = load_tools(["llm-math"], llm=llm)

# Create agent using Zero-shot ReAct (works well with Gemini)
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Ask a math question
response = agent.invoke("What is the result of (45 / 9) + (7 * 3)?")
print("\nFinal Answer:", response)
print("Agent Response:", response['output'])



> Entering new AgentExecutor chain...
Thought: I need to perform the calculations in the parentheses first, then add the results.  I will use the calculator to help.
Action: Calculator
Action Input: (45 / 9) + (7 * 3)
Observation: Answer: 26.0
Thought:Thought: I now know the final answer
Final Answer: 26

> Finished chain.

Final Answer: {'input': 'What is the result of (45 / 9) + (7 * 3)?', 'output': '26'}
Agent Response: 26


## `wikipedia` based AI Query System

In [ ]:
from dotenv import load_dotenv
load_dotenv()

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import Tool
from langgraph.prebuilt import create_react_agent

# LangChain-Community helpers
from langchain_community.utilities.wikipedia import WikipediaAPIWrapper
from langchain_community.tools.wikipedia.tool import WikipediaQueryRun

# 1) Initialize Gemini
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.7,
)

# 2) Build a Wikipedia-query tool by wrapping WikipediaAPIWrapper → WikipediaQueryRun
wiki_wrapper = WikipediaAPIWrapper()
wiki_runner = WikipediaQueryRun(api_wrapper=wiki_wrapper)

wiki_tool = Tool(
    name="wikipedia_search",
    func=wiki_runner.run,
    description=(
        "Use this tool whenever you need factual information, context, or background from Wikipedia. "
        "Input should be a concise topic or question—e.g. 'New York City population', 'Apollo 11 mission', "
        "'Python programming language history', or 'Marie Curie biography'.\n\n"
        "The output is an excerpt (or summary) from the corresponding Wikipedia article. "
        "You can rely on it to gather definitions, dates, statistics, and high-level overviews. "
        "If the initial excerpt doesn’t fully answer a multi-part query, you may run a follow-up search "
        "with a more specific phrase."
    ),
)

# 3) Create a ReAct-style agent that knows about “wikipedia_search”
agent = create_react_agent(llm, [wiki_tool])

# 4) Ask a question
response = agent.invoke({
    "messages": [("human", "How many people live in India?")]
    # "messages": [("human", "How many people live in India as of 2025?")] # This query require other tools like internet search to answer this accurately
})

print("Final Agent Answer:")
print(response["messages"][-1].content)

Final Agent Answer:
India's population is over 1.4 billion people, making it the most populous country in the world.  More precise figures vary slightly depending on the source and year, but it's currently around 1.428 billion according to the UN.


## Multi-Source AI Query System(`wikipedia + DuckDuckGo`)

In [36]:

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import Tool
from langgraph.prebuilt import create_react_agent

# LangChain‐Community Wikipedia helpers
from langchain_community.utilities.wikipedia import WikipediaAPIWrapper
from langchain_community.tools.wikipedia.tool import WikipediaQueryRun

# DuckDuckGo search (no API key required)
from langchain.tools import DuckDuckGoSearchRun

from dotenv import load_dotenv
load_dotenv()

# 1) Initialize Gemini LLM
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.7,
)

# 2a) Build a Wikipedia‐query tool
wiki_wrapper = WikipediaAPIWrapper()
wiki_runner = WikipediaQueryRun(api_wrapper=wiki_wrapper)

wiki_tool = Tool(
    name="wikipedia_search",
    func=wiki_runner.run,
    description=(
        "Use this tool whenever you need factual background or historical context from Wikipedia. "
        "For example, you might ask: 'India population 2020 census' or 'India demographic history'. "
        "It returns a Wikipedia excerpt or summary for your query."
    ),
)

# 2b) Build a DuckDuckGo Internet‐search tool
ddg_wrapper = DuckDuckGoSearchRun()  # performs a live DuckDuckGo query
ddg_tool = Tool(
    name="internet_search",
    func=ddg_wrapper.run,
    description=(
        "Use this tool to fetch up‐to‐date, real‐time information from the web (via DuckDuckGo). "
        "Input should be a query like 'India population 2025 estimate'. "
        "It returns the top search snippets, which often include population figures or news reports."
    ),
)

# 3) Create a ReAct‐style agent that knows about both tools
agent = create_react_agent(llm, [ddg_tool, wiki_tool])

# 4) Ask the agent for India’s 2025 population
response = agent.invoke({
    "messages": [
        ("human", "How many people live in India as of 2025?")
    ]
})

print("Final Agent Answer:")
print(response["messages"][-1].content)

Final Agent Answer:
Based on the internet search, estimates for India's population in 2025 vary.  One estimate puts it at approximately 1.45 billion, making it the world's most populous country.  Another source gives a figure of 1,462,732,387 as of May 31, 2025.  It's important to note that these are estimates, and the exact number may differ slightly depending on the source and methodology used.


# 04: Creating Custom Tools


LangChain allows you to define and integrate custom tools.

These tools can be simple Python functions wrapped for agent use.


```python
# Define a financial reporting function
def financial_report(company_name: str, revenue: int, expenses: int) -> str:
    """Generate a financial report for a company that calculates net income."""

    net_income = revenue - expenses

    report = f"Financial Report for {company_name}:"
    report += f"Revenue: ${revenue}"
    report += f"Expenses: ${expenses}"
    report += f"Net Income: ${net_income}"
    return report

# Test the function
print(financial_report(company_name="LemonadeStand", revenue=100, expenses=50))
```

## Wrapping Function as a Tool


To use the function with an agent, wrap it with `@tool` decorator.


```python
from langchain_core.tools import tool

@tool
def financial_report(company_name: str, revenue: int, expenses: int) -> str:
    """Generate a financial report for a company that calculates net income."""

    net_income = revenue - expenses

    report = f"Financial Report for {company_name}:"
    report += f"Revenue: ${revenue}"
    report += f"Expenses: ${expenses}"
    report += f"Net Income: ${net_income}"
    return report
```

## Integrating Custom Tool with Agent


You can pass your custom tool to the ReAct agent and use it in a natural language query.


```python
agent = create_react_agent(llm, [financial_report])

messages = agent.invoke({
    "messages": [("human", "TechStack generated made $10 million with $8 million of costs. Generate a financial report.")]
})
print(messages)

# Print the agent's final message
print(messages['messages'][-1].content)
```

## Financial report tool with Gemini LLM and LangGraph agent

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI 
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool

from dotenv import load_dotenv
load_dotenv()

# Define a simple tool to calculate area
@tool
def calculate_area(length: float, width: float) -> float:
    """
    Calculate the area of a rectangle given its length and width.
    """
    return length * width

# Initialize Gemini LLM 
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0
)

# Create the ReAct agent with the calculate_area tool
agent = create_react_agent(llm, [calculate_area])

# Define a user message that describes what they want
user_message = "Can you calculate the area of a rectangle with length 10 meters and width 5 meters?"

# Call the agent with the human message
response = agent.invoke({
    "messages": [("human", user_message)]
})

# Print the full response (optional debug)
print("Agent full response:", response)

# Print the final answer only
print("\nFinal Agent Answer:")
print(response["messages"][-1].content)

Agent full response: {'messages': [HumanMessage(content='Can you calculate the area of a rectangle with length 10 meters and width 5 meters?', additional_kwargs={}, response_metadata={}, id='37fbbdec-6568-46f8-bd08-1f89688ae0de'), AIMessage(content='', additional_kwargs={'function_call': {'name': 'calculate_area', 'arguments': '{"length": 10.0, "width": 5.0}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--0facb4f5-8f54-48c8-afd5-4b96f4d07737-0', tool_calls=[{'name': 'calculate_area', 'args': {'length': 10.0, 'width': 5.0}, 'id': 'c97a715f-71e7-42e1-bf8b-f3c96f5b745a', 'type': 'tool_call'}], usage_metadata={'input_tokens': 42, 'output_tokens': 7, 'total_tokens': 49, 'input_token_details': {'cache_read': 0}}), ToolMessage(content='50.0', name='calculate_area', id='ed2202f0-4b32-43b8-a3ae-0eb2fa04aa36', tool_call_id='c97a715f-71e7-42e1-bf8b-f3c96f5b745a'), AIMessage(content='The area of the recta

In [ ]:

from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool

from dotenv import load_dotenv
load_dotenv()

# Define the financial return report
# The @tool decorator registers this function as a tool for the LangChain agent
@tool
def financial_report(company_name: str, revenue: int, expenses: int) -> str:
    
    net_income = revenue - expenses
    
    report = f"Financial Report for {company_name}:\n"
    report += f"Revenue: ${revenue:,}\n"
    report += f"Expenses: ${expenses:,}\n"
    report += f"Net Income: ${net_income:,}\n"
    
    if net_income > 0:
        report += "Status: Profitable\n"
    elif net_income < 0:
        report += "Status: Loss\n"
    else:
        report += "Status: Break-even\n"
    
    return report

# Initialize Gemini LLM 
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash",
                             temperature=0)

# Create the ReAct agent with the financial_report tool
agent = create_react_agent(llm, [financial_report])

# Define input message
user_message = "TechStack generated made $10 million with $8 million of costs. Generate a financial report."

# Call the agent with the human message
response = agent.invoke({
    "messages": [("human", user_message)]
})

# Print the full response (including reasoning steps if verbose is enabled)
print("Agent full response:", response)

# Print the final answer content only
print("\nFinal Agent Answer:")
print(response["messages"][-1].content)

Agent full response: {'messages': [HumanMessage(content='TechStack generated made $10 million with $8 million of costs. Generate a financial report.', additional_kwargs={}, response_metadata={}, id='99edd5f9-2ec1-4bbf-960d-999bde80e223'), AIMessage(content='', additional_kwargs={'function_call': {'name': 'financial_report', 'arguments': '{"revenue": 10000000.0, "expenses": 8000000.0, "company_name": "TechStack"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run--abb3b8e7-d4a2-4595-bcbb-c8623c4ff5b3-0', tool_calls=[{'name': 'financial_report', 'args': {'revenue': 10000000.0, 'expenses': 8000000.0, 'company_name': 'TechStack'}, 'id': '1e81cad7-66a6-4cdb-bd69-a40f75744fc3', 'type': 'tool_call'}], usage_metadata={'input_tokens': 90, 'output_tokens': 12, 'total_tokens': 102, 'input_token_details': {'cache_read': 0}}), ToolMessage(content='Financial Report for TechStack:\nRevenue: $10,000,000\nExpenses